# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Daily Prediction_Step by Step/sales_train_validation.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks_Daily Prediction_Step by Step/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Parameters settings

In [7]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100

In [8]:
validation_training_ratio = 0.99

In [9]:
list_params = list()
for p in [0, 1, 2, 3]:
    for d in range(2):
        for q in range(4):
            list_params.append((p, d, q))

In [10]:
list_params

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 0, 3),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (3, 0, 0),
 (3, 0, 1),
 (3, 0, 2),
 (3, 0, 3),
 (3, 1, 0),
 (3, 1, 1),
 (3, 1, 2),
 (3, 1, 3)]

In [11]:
len(list_params)

32

# This step involves specifying Demand Pattern (Intermittent or Lumpy or Erratic or Smooth)

In [12]:
sales_pattern = sales_smooth.copy()

In [13]:
sales_pattern_py = sales_pattern.copy()
list_pattern_py = sales_pattern_py.id.unique().tolist()

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
test_data = df_pattern.iloc[n_pred_days*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

# Hyperparameter for ARIMA model

In [14]:
list_sample = sample(list_pattern_py, 50)

In [15]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in list_sample:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_sample.index(product) / len(list_sample) * 100, 2)))
    train_first = train_data_first_part[product].iloc[-num_train_needed:]
    train_second = train_data_second_part[product]
    
    for combi in list_params:
        model = ARIMA(train_first, order = combi)
        model_fit = model.fit()
        forecast_temp = model_fit.forecast(len(train_second))
        df_temp = pd.DataFrame({'Date': train_second.index,
                                'Actual Data': train_second.values,
                                'Forecast': forecast_temp.values,
                                'Combination': [combi for combi_count in range(len(train_second))], 
                                'Product': [product for product_count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_temp, ignore_index=False)
        
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_3_714_CA_3_validation
Progressing: 0.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_694_CA_1_validation
Progressing: 2.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_227_CA_4_validation
Progressing: 4.0 %


...

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_406_CA_3_validation
Progressing: 98.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimiz

This line of code took 6.158912339066666 minutes


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [16]:
df_result_all_params

,Date,Actual Data,Forecast,Combination,Product
0,2016-04-03,42.0,30.409995,"(0, 0, 0)",FOODS_3_714_CA_3_validation
1,2016-04-04,30.0,30.409995,"(0, 0, 0)",FOODS_3_714_CA_3_validation
2,2016-04-05,40.0,30.409995,"(0, 0, 0)",FOODS_3_714_CA_3_validation
3,2016-04-06,57.0,30.409995,"(0, 0, 0)",FOODS_3_714_CA_3_validation
4,2016-04-07,42.0,30.409995,"(0, 0, 0)",FOODS_3_714_CA_3_validation
...,...,...,...,...,...
3,2016-04-06,5.0,14.721580,"(3, 1, 3)",FOODS_3_406_CA_3_validation
4,2016-04-07,28.0,14.939360,"(3, 1, 3)",FOODS_3_406_CA_3_validation
5,2016-04-08,17.0,16.698024,"(3, 1, 3)",FOODS_3_406_CA_3_validation
6,2016-04-09,28.0,14.364262,"(3, 1, 3)",FOODS_3_406_CA_3_validation


# Summay all metrics

In [17]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [18]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Combination']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.20976193044999908 minutes


In [19]:
df_result_metrics_all_params

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_3_validation,"(0, 0, 0)",25.987482,5939.9960,194.1520,654.9995,7.424995,7.432357
1,FOODS_1_004_CA_3_validation,"(0, 0, 1)",24.766844,5660.9930,194.1115,649.1091,7.076241,7.120266
2,FOODS_1_004_CA_3_validation,"(0, 0, 2)",24.324825,5559.9601,193.8373,611.3296,6.949950,7.018304
3,FOODS_1_004_CA_3_validation,"(0, 0, 3)",22.489654,5140.4924,192.0779,431.1485,6.425615,6.592962
4,FOODS_1_004_CA_3_validation,"(0, 1, 0)",0.437500,100.0000,NaN,100.0000,0.125000,0.353553
...,...,...,...,...,...,...,...,...
1595,FOODS_3_811_CA_4_validation,"(3, 0, 3)",1.501593,102.9663,103.5178,116.3309,1.930619,2.076675
1596,FOODS_3_811_CA_4_validation,"(3, 1, 0)",1.229441,84.3045,87.5020,73.5436,1.580710,1.836675
1597,FOODS_3_811_CA_4_validation,"(3, 1, 1)",1.230790,84.3970,86.6249,79.8119,1.582444,1.883728
1598,FOODS_3_811_CA_4_validation,"(3, 1, 2)",1.185744,81.3082,85.9096,80.1399,1.524529,1.820787


# Check what products has unexpected metrics result

In [20]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
160,FOODS_2_050_CA_2_validation,"(0, 0, 0)",inf,inf,200.0,NaN,0.000005,0.000005
161,FOODS_2_050_CA_2_validation,"(0, 0, 1)",inf,inf,200.0,NaN,0.000005,0.000005
162,FOODS_2_050_CA_2_validation,"(0, 0, 2)",inf,inf,200.0,NaN,0.000005,0.000005
163,FOODS_2_050_CA_2_validation,"(0, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
164,FOODS_2_050_CA_2_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1339,FOODS_3_661_CA_4_validation,"(3, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
1340,FOODS_3_661_CA_4_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
1341,FOODS_3_661_CA_4_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000000,0.000000
1342,FOODS_3_661_CA_4_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000000,0.000000


In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
160,FOODS_2_050_CA_2_validation,"(0, 0, 0)",inf,inf,200.0,NaN,0.000005,0.000005
161,FOODS_2_050_CA_2_validation,"(0, 0, 1)",inf,inf,200.0,NaN,0.000005,0.000005
162,FOODS_2_050_CA_2_validation,"(0, 0, 2)",inf,inf,200.0,NaN,0.000005,0.000005
163,FOODS_2_050_CA_2_validation,"(0, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
164,FOODS_2_050_CA_2_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1339,FOODS_3_661_CA_4_validation,"(3, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
1340,FOODS_3_661_CA_4_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
1341,FOODS_3_661_CA_4_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000000,0.000000
1342,FOODS_3_661_CA_4_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000000,0.000000


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
4,FOODS_1_004_CA_3_validation,"(0, 1, 0)",0.4375,100.0,NaN,100.0,0.125,0.353553
164,FOODS_2_050_CA_2_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000,0.000000
165,FOODS_2_050_CA_2_validation,"(0, 1, 1)",NaN,NaN,NaN,NaN,0.000,0.000000
166,FOODS_2_050_CA_2_validation,"(0, 1, 2)",NaN,NaN,NaN,NaN,0.000,0.000000
167,FOODS_2_050_CA_2_validation,"(0, 1, 3)",NaN,NaN,NaN,NaN,0.000,0.000000
...,...,...,...,...,...,...,...,...
1335,FOODS_3_661_CA_4_validation,"(2, 1, 3)",NaN,NaN,NaN,NaN,0.000,0.000000
1340,FOODS_3_661_CA_4_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000,0.000000
1341,FOODS_3_661_CA_4_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000,0.000000
1342,FOODS_3_661_CA_4_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000,0.000000


In [23]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
160,FOODS_2_050_CA_2_validation,"(0, 0, 0)",inf,inf,200.0,NaN,0.000005,0.000005
161,FOODS_2_050_CA_2_validation,"(0, 0, 1)",inf,inf,200.0,NaN,0.000005,0.000005
162,FOODS_2_050_CA_2_validation,"(0, 0, 2)",inf,inf,200.0,NaN,0.000005,0.000005
163,FOODS_2_050_CA_2_validation,"(0, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
164,FOODS_2_050_CA_2_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1339,FOODS_3_661_CA_4_validation,"(3, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
1340,FOODS_3_661_CA_4_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
1341,FOODS_3_661_CA_4_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000000,0.000000
1342,FOODS_3_661_CA_4_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000000,0.000000


In [24]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [25]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [26]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [27]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 10.0 %
Percentage of unexpected values of WMAPE is: 10.0 %
Percentage of unexpected values of SMAPE is: 26.0 %
Percentage of unexpected values of MAPE is: 10.0 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [28]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Combination

In [29]:
df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Combination,,,,,,
"(0, 0, 0)",1.441346,194.861427,84.043873,71.728100,3.169611,3.954962
"(0, 0, 1)",1.416549,188.772876,84.193929,72.063962,3.174991,3.966688
"(0, 0, 2)",1.405965,186.409884,84.102562,71.215824,3.167183,3.950812
"(0, 0, 3)",1.369589,177.444824,84.012211,67.018676,3.173973,3.925695
"(0, 1, 0)",1.153350,76.908222,92.411851,78.427273,4.462838,5.322598
"(0, 1, 1)",0.962901,79.572320,84.229643,59.467939,3.226699,4.044480
"(0, 1, 2)",1.034002,94.778586,84.487973,61.925391,3.234364,4.039097
"(0, 1, 3)",1.020457,93.171786,83.965173,61.625625,3.199498,3.986209
"(1, 0, 0)",1.318907,166.661091,84.167538,65.838893,3.146869,3.947902


In [30]:
for mts in list_metrics:
    print('The optimum Combination based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Combination based on Mean MASE is: (2, 1, 1)
The optimum Combination based on Mean WMAPE is: (1, 1, 0)
The optimum Combination based on Mean SMAPE is: (2, 1, 2)
The optimum Combination based on Mean MAPE is: (0, 1, 1)
The optimum Combination based on Mean MAE is: (3, 0, 3)
The optimum Combination based on Mean RMSE is: (3, 0, 3)


### Get MEDIAN metrics value of each Combination

In [31]:
df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Combination,,,,,,
"(0, 0, 0)",0.757647,61.53850,74.00960,56.51690,2.200002,2.585654
"(0, 0, 1)",0.752055,60.78050,73.78920,57.28720,2.261778,2.591789
"(0, 0, 2)",0.757193,64.62900,73.30710,57.17310,2.220046,2.577190
"(0, 0, 3)",0.767871,63.26220,72.40180,57.24890,2.201007,2.576096
"(0, 1, 0)",1.000000,71.42860,87.38100,63.02580,2.625000,3.061862
"(0, 1, 1)",0.819965,60.73530,73.73235,56.14475,2.003651,2.476827
"(0, 1, 2)",0.809532,60.11385,73.28515,57.13160,2.051222,2.514059
"(0, 1, 3)",0.809017,60.66510,72.55685,57.32850,2.097046,2.554639
"(1, 0, 0)",0.761379,63.37350,73.69200,57.26650,2.266355,2.593466


In [32]:
for mts in list_metrics:
    print('The optimum Combination based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Combination based on Median of MASE is: (0, 0, 1)
The optimum Combination based on Median of WMAPE is: (1, 1, 1)
The optimum Combination based on Median of SMAPE is: (3, 0, 1)
The optimum Combination based on Median of MAPE is: (2, 1, 3)
The optimum Combination based on Median of MAE is: (0, 1, 1)
The optimum Combination based on Median of RMSE is: (0, 1, 1)


# Run ARIMA model after decide best paramaters

In [33]:
best_combination = (0, 1, 1)

In [34]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    train_product = train_data[product].iloc[num_train_needed*-1:]
    test_product = test_data[product]  
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()

    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Date': test_product.index,
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values,
                                   'Product': [product for product_count in range(len(test_product))]})
    
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_1_004_CA_1_validation
Progressing: 0.0 %
Currently Running Product: FOODS_1_012_CA_1_validation
Progressing: 0.16 %
Currently Running Product: FOODS_1_018_CA_1_validation
Progressing: 0.31 %
Currently Running Product: FOODS_1_043_CA_1_validation
Progressing: 0.47 %
Currently Running Product: FOODS_1_046_CA_1_validation
Progressing: 0.62 %
Currently Running Product: FOODS_1_085_CA_1_validation
Progressing: 0.78 %
Currently Running Product: FOODS_1_086_CA_1_validation
Progressing: 0.93 %
Currently Running Product: FOODS_1_087_CA_1_validation
Progressing: 1.09 %
Currently Running Product: FOODS_1_097_CA_1_validation
Progressing: 1.25 %
Currently Running Product: FOODS_1_099_CA_1_validation
Progressing: 1.4 %
Currently Running Product: FOODS_1_130_CA_1_validation
Progressing: 1.56 %
Currently Running Product: FOODS_1_139_CA_1_validation
Progressing: 1.71 %
Currently Running Product: FOODS_1_153_CA_1_validation
Progressing: 1.87 %
Currently Running Product: 

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_2_197_CA_1_validation
Progressing: 5.61 %
Currently Running Product: FOODS_2_222_CA_1_validation
Progressing: 5.76 %
Currently Running Product: FOODS_2_233_CA_1_validation
Progressing: 5.92 %
Currently Running Product: FOODS_2_244_CA_1_validation
Progressing: 6.07 %
Currently Running Product: FOODS_2_276_CA_1_validation
Progressing: 6.23 %
Currently Running Product: FOODS_2_299_CA_1_validation
Progressing: 6.39 %
Currently Running Product: FOODS_2_325_CA_1_validation
Progressing: 6.54 %
Currently Running Product: FOODS_2_347_CA_1_validation
Progressing: 6.7 %
Currently Running Product: FOODS_2_349_CA_1_validation
Progressing: 6.85 %
Currently Running Product: FOODS_2_357_CA_1_validation
Progressing: 7.01 %
Currently Running Product: FOODS_2_359_CA_1_validation
Progressing: 7.17 %
Currently Running Product: FOODS_2_364_CA_1_validation
Progressing: 7.32 %
Currently Running Product: FOODS_2_371_CA_1_validation
Progressing: 7.48 %
Currently Running Product:

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_032_CA_1_validation
Progressing: 8.1 %
Currently Running Product: FOODS_3_034_CA_1_validation
Progressing: 8.26 %
Currently Running Product: FOODS_3_042_CA_1_validation
Progressing: 8.41 %
Currently Running Product: FOODS_3_053_CA_1_validation
Progressing: 8.57 %
Currently Running Product: FOODS_3_054_CA_1_validation
Progressing: 8.72 %
Currently Running Product: FOODS_3_064_CA_1_validation
Progressing: 8.88 %
Currently Running Product: FOODS_3_067_CA_1_validation
Progressing: 9.03 %
Currently Running Product: FOODS_3_076_CA_1_validation
Progressing: 9.19 %
Currently Running Product: FOODS_3_080_CA_1_validation
Progressing: 9.35 %
Currently Running Product: FOODS_3_083_CA_1_validation
Progressing: 9.5 %
Currently Running Product: FOODS_3_086_CA_1_validation
Progressing: 9.66 %
Currently Running Product: FOODS_3_087_CA_1_validation
Progressing: 9.81 %
Currently Running Product: FOODS_3_090_CA_1_validation
Progressing: 9.97 %
Currently Running Product: 

...

Currently Running Product: FOODS_3_766_CA_3_validation
Progressing: 81.78 %
Currently Running Product: FOODS_3_768_CA_3_validation
Progressing: 81.93 %
Currently Running Product: FOODS_3_782_CA_3_validation
Progressing: 82.09 %
Currently Running Product: FOODS_3_789_CA_3_validation
Progressing: 82.24 %
Currently Running Product: FOODS_3_792_CA_3_validation
Progressing: 82.4 %
Currently Running Product: FOODS_3_793_CA_3_validation
Progressing: 82.55 %
Currently Running Product: FOODS_3_794_CA_3_validation
Progressing: 82.71 %
Currently Running Product: FOODS_3_795_CA_3_validation
Progressing: 82.87 %
Currently Running Product: FOODS_3_797_CA_3_validation
Progressing: 83.02 %
Currently Running Product: FOODS_3_804_CA_3_validation
Progressing: 83.18 %
Currently Running Product: FOODS_3_811_CA_3_validation
Progressing: 83.33 %
Currently Running Product: FOODS_3_818_CA_3_validation
Progressing: 83.49 %
Currently Running Product: FOODS_3_819_CA_3_validation
Progressing: 83.64 %
Currently Run

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_661_CA_4_validation
Progressing: 96.57 %
Currently Running Product: FOODS_3_668_CA_4_validation
Progressing: 96.73 %
Currently Running Product: FOODS_3_672_CA_4_validation
Progressing: 96.88 %
Currently Running Product: FOODS_3_681_CA_4_validation
Progressing: 97.04 %
Currently Running Product: FOODS_3_694_CA_4_validation
Progressing: 97.2 %
Currently Running Product: FOODS_3_695_CA_4_validation
Progressing: 97.35 %
Currently Running Product: FOODS_3_697_CA_4_validation
Progressing: 97.51 %
Currently Running Product: FOODS_3_704_CA_4_validation
Progressing: 97.66 %
Currently Running Product: FOODS_3_708_CA_4_validation
Progressing: 97.82 %
Currently Running Product: FOODS_3_714_CA_4_validation
Progressing: 97.98 %
Currently Running Product: FOODS_3_723_CA_4_validation
Progressing: 98.13 %
Currently Running Product: FOODS_3_732_CA_4_validation
Progressing: 98.29 %
Currently Running Product: FOODS_3_739_CA_4_validation
Progressing: 98.44 %
Currently Run

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_807_CA_4_validation
Progressing: 99.53 %
Currently Running Product: FOODS_3_811_CA_4_validation
Progressing: 99.69 %
Currently Running Product: FOODS_3_818_CA_4_validation
Progressing: 99.84 %
This line of code took 1.2227631703999995 minutes


In [35]:
df_result_best_params

,Date,Actual Data,Forecast,Product
0,2016-04-11,2.0,4.718816,FOODS_1_004_CA_1_validation
1,2016-04-12,11.0,4.718816,FOODS_1_004_CA_1_validation
2,2016-04-13,8.0,4.718816,FOODS_1_004_CA_1_validation
3,2016-04-14,0.0,4.718816,FOODS_1_004_CA_1_validation
4,2016-04-15,10.0,4.718816,FOODS_1_004_CA_1_validation
...,...,...,...,...
8983,2016-04-20,4.0,2.141899,FOODS_3_818_CA_4_validation
8984,2016-04-21,1.0,2.141899,FOODS_3_818_CA_4_validation
8985,2016-04-22,2.0,2.141899,FOODS_3_818_CA_4_validation
8986,2016-04-23,3.0,2.141899,FOODS_3_818_CA_4_validation


In [36]:
#df_result_best_params.to_csv('ARIMA_Smooth_Point_Forecast.csv')

In [37]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [38]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_1_validation,0.700296,61.4808,72.7284,63.5768,4.040169,4.909750
1,FOODS_1_004_CA_3_validation,inf,inf,200.0000,NaN,0.009450,0.009450
2,FOODS_1_004_CA_4_validation,0.961351,57.8551,60.2932,86.0264,1.405051,1.731062
3,FOODS_1_012_CA_1_validation,0.662474,46.1634,53.5986,73.6530,1.681665,2.080806
4,FOODS_1_012_CA_2_validation,0.651009,38.2830,42.2447,37.0076,4.156444,5.091203
...,...,...,...,...,...,...,...
637,FOODS_3_818_CA_4_validation,0.550243,40.8669,46.2101,41.1732,0.846528,1.101885
638,FOODS_3_819_CA_3_validation,0.835922,67.0755,59.9655,146.6761,2.443463,2.926037
639,FOODS_3_820_CA_1_validation,0.546884,51.7564,65.5337,41.2806,1.219971,1.491287
640,FOODS_3_820_CA_3_validation,0.656459,67.7499,84.7203,38.1833,1.161428,1.393778


In [39]:
df_result_final.RMSE.mean()

3.739865970155693

In [40]:
df_result_final.MAPE.mean()

66.85309717314487

In [41]:
df_result_final.to_csv('ARIMA_Smooth_Metrics.csv')